In [2]:
import pymssql
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)
import pandas as pd
from tinydb import TinyDB, Query
import json

databse_only_important_sample = "./database_only_important_sample/"
databse_full_sample = "./database_full_sample/"
database_only_important_sample_json = "./database_only_important_sample_json/"
databse_tiny_db = "./database_tiny/"

def write_to_full(df,  name):
    df = df.map(lambda x: x.strip() if isinstance(x, str) else x)
    df.to_json(databse_full_sample + name+ ".json", orient="records", indent=4)

def write_to_important(df,  name):
    df.to_csv(databse_only_important_sample + name+ ".csv")

def write_to_json(df,  name):
    df = df.map(lambda x: x.strip() if isinstance(x, str) else x)
    df = df.map(lambda x: x[:500] if isinstance(x, str) and len(x) > 75 else x)
    # df.to_json(database_only_important_sample_json + name+ ".json", orient="records", indent=4)
    data_json = json.loads(df.to_json(orient="records"))
    tiny_db = TinyDB("./database_tiny/" + name+".json")
    for i in data_json:
        tiny_db.insert(i)
    
conn = pymssql.connect(server="DESKTOP-0MUAA95\BPSINSTANCE" , user="bpsrawdata", password="password", database="BPSSamples") 
cursor = conn.cursor()

In [6]:
Table_name = ["PATIENTS", "USERS", "Appointments", "APPOINTMENTTYPES", "APPOINTMENTCODES","VISITS", "VISITTYPE", "VISITREASON"]
for i in Table_name:
    df = pd.read_sql("SELECT * FROM " + i, conn) 
    write_to_full(df, i)


In [12]:
df = pd.read_sql("SELECT * FROM PATIENTS", conn) 
write_to_full(df, "Patients")


In [3]:
df = pd.read_sql('''SELECT INTERNALID
    ,[FIRSTNAME]
      ,[MIDDLENAME]
      ,[SURNAME]
      ,[PREFERREDNAME]       
      ,[ADDRESS1]
      ,[ADDRESS2]
      ,[CITY]
      ,[POSTCODE]
        ,[DOB]
        ,[HOMEPHONE]
      ,[WORKPHONE]
      ,[MOBILEPHONE]
        ,[MEDICARENO]
      
      FROM PATIENTS
      
      ''', conn)
 
# write_to_important(df, "Patients")
write_to_json(df, "Patients")

In [10]:
df = pd.read_sql('''
    SELECT [USERID]
    ,[SURNAME]
      ,[FIRSTNAME]      
      ,[EMAIL]      
      ,[APPOINTMENTS]
      FROM USERS''', conn) 
# write_to_important(df, "GPs_info")
write_to_json(df, "GPs_info")

In [11]:
df = pd.read_sql('''
    SELECT  app.[RECORDID]
      ,app.INTERNALID 
      ,PATIENTS.FIRSTNAME as PATIENTS_FIRSTNAME
	,PATIENTS.SURNAME as PATIENTS_SURNAME
      ,app.USERID
      ,USERS.FIRSTNAME as users_firstname
      ,USERs.SURNAME as users_SURNAME
      ,app.[APPOINTMENTDATE]
      ,app.[APPOINTMENTTIME]
      ,app.[APPOINTMENTLENGTH]
      ,code.DESCRIPTION as app_code
      ,typ.[DESCRIPTION] as app_type
      ,app.[REASON]   
      ,app.[ARRIVALTIME]
      ,app.[CONSULTATIONTIME]
      ,app.[ROOMID]
      ,app.[CREATED]
      ,app.[CREATEDBY]
      ,app.[BOOKEDBY]
      ,app.[URGENT]      
      ,app.[ITEMLIST]
      ,app.[COMMENT]
      FROM Appointments as app
	  join PATIENTS on app.INTERNALID = PATIENTS.INTERNALID
	  join USERS on app.USERID = USERS.USERID
        join APPOINTMENTTYPES as typ on app.APPOINTMENTTYPE = typ.APPOINTMENTCODE
	  join APPOINTMENTCODES as code on app.APPOINTMENTCODE = code.APPOINTMENTCODE
      
      
      ''', conn) 
# write_to_important(df, "Appointments")
write_to_json(df, "Appointments")

In [12]:
df = pd.read_sql('''
SELECT   visit.[VISITID]
      ,visit.INTERNALID
      ,PATIENTS.FIRSTNAME as PATIENTS_FIRSTNAME
	,PATIENTS.SURNAME as PATIENTS_SURNAME
      ,visit.USERID
      ,USERS.FIRSTNAME as users_firstname
	  ,USERs.SURNAME as users_SURNAME
      ,visit.[RECORDSTATUS]
      ,typ.VISITTYPE
      ,visit.[DRNAME]
      ,visit.[VISITDATE]
      ,visit.[DURATION]
      ,visit.[VISITNOTES]
	,reason.REASON
  FROM VISITS as visit
  	  join USERS on visit.USERID = USERS.USERID
join PATIENTS on visit.INTERNALID = PATIENTS.INTERNALID
  join VISITTYPE as typ on visit.VISITCODE = typ.VISITCODE
  join VISITREASON as reason on visit.VISITID = reason.VISITID
      
      
      ''', conn) 
# write_to_important(df, "Patients_visits")
write_to_json(df, "Patients_visits")


In [4]:
df = pd.read_sql('''
SELECT ph.[RECORDID]
      ,ph.[RECORDSTATUS]
      ,ph.INTERNALID
      ,PATIENTS.FIRSTNAME as PATIENTS_FIRSTNAME
	,PATIENTS.SURNAME as PATIENTS_SURNAME
      ,ph.[YEAR]
      ,ph.[MONTH]
      ,ph.[DAY]
      ,ph.[ITEMTEXT]
      ,ph.[ITEMCODE]
      ,ph.[STATUSCODE]
      ,ph.[DETAILS]
  FROM  [PASTHISTORY] as ph
  join PATIENTS on ph.INTERNALID = PATIENTS.INTERNALID
      ''', conn) 
write_to_json(df, "Patients_medication_history")


In [ ]:


conn.close()